In [201]:
#import requited libraries 
from requests import get
from bs4 import BeautifulSoup
from xlsxwriter import Workbook
import xlrd
import pandas as pd
import re
from datetime import date

In [202]:
#check current working directory 
import os
print(os.getcwd())

C:\Users\malejin\Documents\_diploma\test_bs


In [203]:
#import pyodbc library
import pyodbc

In [204]:
#check for drivers
for driver in pyodbc.drivers():
    print(driver)

SQL Server
Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)


In [205]:
#set connection
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-42IUD2D;'
                      'Database=Pharmacy_Price_Comparsion;'
                      'Trusted_Connection=yes;')

In [206]:
cursor = conn.cursor()

In [207]:
#import ogyi_product_names from dbo.ogyi_product_list_simplified
ogyi_workbook = pd.read_sql_query('''SELECT * from dbo.ogyi_product_list_simplified''', conn)
type(ogyi_workbook)

pandas.core.frame.DataFrame

In [208]:
ogyi_workbook.columns[1]
ogyi_product_name=ogyi_workbook[ogyi_workbook.columns[1]]
type(ogyi_product_name)
len(ogyi_product_name)  
ogyi_product_name  
    

0              ACC 100 mg granulátum gyermekeknek
1                   ACC 20 mg/ml belsőleges oldat
2                           ACC 200 mg granulátum
3                       ACC 200 mg pezsgőtabletta
4          ACC HOT 600 mg por belsõleges oldathoz
                          ...                    
233                       XYZAL 5 mg filmtabletta
234                    YOSQIERO 5 mg filmtabletta
235                      ZILOLA 5 mg filmtabletta
236                     ZYRTEC 10 mg filmtabletta
237    ZYRTEC 10 mg/ml belsőleges oldatos cseppek
Name: ogyi_product_name, Length: 238, dtype: object

In [209]:
ogyi_product_name[0]


'ACC 100 mg granulátum gyermekeknek'

In [210]:
#set url base which is the search page url of BENU. I am using this due to the complexity of the product names. There is no exact rules for the details page url setup, also i would need to use 3columns of the OGYI site but no exact rules can be defined which string parts can compose the exact search url..So most probably this way is more efficient.
url_base='https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q='
url_base


'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q='

In [211]:
#generate the search page urls
url_list=[]
for i in range(len(ogyi_workbook)):
    url_list.append(url_base+ogyi_product_name[i])

url_list

['https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=ACC 100 mg granulátum gyermekeknek',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=ACC 20 mg/ml belsőleges oldat',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=ACC 200 mg granulátum',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=ACC 200 mg pezsgőtabletta',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=ACC HOT 600 mg por belsõleges oldathoz',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=ACC HOT 600 mg por belsőleges oldathoz',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=ACC INSTANT 600 mg belsőleges por',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=ACC LONG 600 mg pezsgőtabletta',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=AFRIN 0,5 mg/ml oldatos orrspray',
 'https://benu.hu/shop/catalogsearch/result/?wp-kereses=1&q=AFRIN COMFORT MENTOLLAL 0,5 mg/ml oldatos orrspray',
 'https://benu.hu/shop/catalogsearch/res

In [212]:
#check the len of the url list
type(url_list[0])

str

In [213]:
#change iopub_data_rate_limit in prompt
#NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)-> I changed to NotebookApp.iopub_data_rate_limit = 100000000
#NotebookApp.rate_limit_window=3.0 (secs)-> I changed it to NotebookApp.rate_limit_window
#also buffering limit was changed

In [214]:
#generate the soupf of the content of the url list
soup_list=[]

from timeit import default_timer as timer

start = timer()

soup_list.append(BeautifulSoup(get(url_list[i]).text))

end = timer()


In [215]:
#check how much time the souping takes
print(end - start) #time in seconds

#souping 150urls takes 48.15 secs if printing included. Without pringting it takes 43.90
#souping 300urls takes 94.16 secs without printing
#souping 625 urls takes 187.73 secs without printing

0.43728820000023916


In [216]:
#check the soup
soup_list[0]
type(soup_list[0])
len(soup_list)

1

In [217]:
#validate that those urls has no 'div', {'class': 'wrapper-hover product-box' where the product was not found
searched_product_s0=soup_list[0].find_all('div', {'class': 'wrapper-hover product-box'})
print(searched_product_s0)

[<div class="wrapper-hover product-box">
<a class="product-image" href="https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml" itemprop="url" onclick="window.prefixboxAnalytics('click', 'serp', {'pattern':'ZYRTEC 10 mg/ml belsőleges oldatos cseppek', 'position':1, 'productId':'44922', 'title':'Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml', 'url':'https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml', 'category':'Allergia'})" rel="nofollow" title="Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml">
<img alt="Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml" itemprop="image" src="https://benu.hu/shop/media/catalog/product/cache/1/small_image/130x130/9df78eab33525d08d6e5fb8d27136e95/Z/y/Zyrtec_10_mg_ml_belsoleges_oldatos_cseppek__20ml_225971_2016_1.jpg"/>
</a>
<div class="product-shop">
<h2 class="product-name">
<a href="https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml" itemprop="name" onclick="window.prefixboxAnalytics('click', 'serp', {

In [218]:
#save only those page contents where product so 'div', {'class': 'wrapper-hover product-box' was found
#since we search for an exact product name the result page will list only 1 product as i found so 'find' looks sufficient
searched_product_all=[]
product_not_found=[]
souped_product_names=[]
souped_product_urls=[]

start = timer()

for i in range(len(soup_list)):
    soup_of_product_boxes=soup_list[i].find('div', {'class': 'wrapper-hover product-box'})
    if soup_of_product_boxes != None:
        searched_product_all.append(soup_of_product_boxes)
        souped_product_names.append(ogyi_product_name[i])
        souped_product_urls.append(url_list[i])
    else: product_not_found.append(url_list[i])
    #here still i would like to add the url of the product that was not found
end = timer()
#print(end - start)#time in seconds

In [219]:
souped_product_names

['ACC 100 mg granulátum gyermekeknek']

In [220]:
#check how much it took to filter out the soup
print(end - start)#time in seconds

0.014456399999744463


In [221]:
print(searched_product_all)

[<div class="wrapper-hover product-box">
<a class="product-image" href="https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml" itemprop="url" onclick="window.prefixboxAnalytics('click', 'serp', {'pattern':'ZYRTEC 10 mg/ml belsőleges oldatos cseppek', 'position':1, 'productId':'44922', 'title':'Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml', 'url':'https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml', 'category':'Allergia'})" rel="nofollow" title="Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml">
<img alt="Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml" itemprop="image" src="https://benu.hu/shop/media/catalog/product/cache/1/small_image/130x130/9df78eab33525d08d6e5fb8d27136e95/Z/y/Zyrtec_10_mg_ml_belsoleges_oldatos_cseppek__20ml_225971_2016_1.jpg"/>
</a>
<div class="product-shop">
<h2 class="product-name">
<a href="https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml" itemprop="name" onclick="window.prefixboxAnalytics('click', 'serp', {

In [222]:
#check length of searched_product_all as it shows how many product was actually found
len(searched_product_all)

1

In [223]:
#checking the data
searched_product_all

[<div class="wrapper-hover product-box">
 <a class="product-image" href="https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml" itemprop="url" onclick="window.prefixboxAnalytics('click', 'serp', {'pattern':'ZYRTEC 10 mg/ml belsőleges oldatos cseppek', 'position':1, 'productId':'44922', 'title':'Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml', 'url':'https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml', 'category':'Allergia'})" rel="nofollow" title="Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml">
 <img alt="Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml" itemprop="image" src="https://benu.hu/shop/media/catalog/product/cache/1/small_image/130x130/9df78eab33525d08d6e5fb8d27136e95/Z/y/Zyrtec_10_mg_ml_belsoleges_oldatos_cseppek__20ml_225971_2016_1.jpg"/>
 </a>
 <div class="product-shop">
 <h2 class="product-name">
 <a href="https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml" itemprop="name" onclick="window.prefixboxAnalytics('click', 'se

In [224]:
#filter to a product name
benu_product_name=[]
for i in range(len(searched_product_all)):
    benu_product_name.append(searched_product_all[i].h2.a.get_text('title').strip())
len(benu_product_name)

1

In [225]:
#filter to price of a product
searched_product_all[0].div.div.get('content')

price=[]
for i in range(len(searched_product_all)):
    if searched_product_all[i].div.div.get('content')!=None:
        price.append(str(searched_product_all[i].div.div.get('content')))
    else:price.append("üres")
price


['1549']

In [226]:
#Remove html tags own function. This will be used by each section later in the code.
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

#Clean string after html removal
def cleanstring(data):
    for i in range(len(data)):
        if data!="üres":
            data=data.strip().replace('\n', ' ').replace('\\r', ' ').replace(u'\xa0',' ').replace("\\xa0","").replace("  "," ").replace(",,",",").replace(", ,",",").replace(" , ",",").replace('\t', ' ').replace("amp;","").replace(" :",":").replace(" ,",",")

            while data.endswith("]']"):
                data=re.sub(r"]']+$","", str(data)).strip()
                
            while data.startswith("['"):
                data=re.sub(r"^\['+", "", str(data)).strip()
            
            while data.endswith(("'",",")):
                data=re.sub(r"'|,+$","", str(data)).strip()
                
            while data.startswith((",",":")):
                data=re.sub(r"^\'|^\,|^\:+", "", str(data)).strip() 
            

    return data   

#removal of unwanted copy parts
def removeunwantedparts(data):
    for i in range(len(data)):
        if data!="üres":
            if 'A forgalomba hozatali engedély jogosultja' in data:
                data=data.replace('A forgalomba hozatali engedély jogosultja',"").strip()
            if 'A forgalombahozatali engedély jogosultja' in data:
                data=data.replace('A forgalombahozatali engedély jogosultja',"").strip()
            if 'és gyártó' in data:
                data=data.replace('és gyártó',"").strip()
            if 'és a gyártó' in data:
                data=data.replace('és a gyártó',"").strip()
            if 'Gyártó' in data:
                data=data.replace('Gyártó',"").strip()
            if "A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjának helyi képviseletéhez" in data:
                data=data.replace("A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjának helyi képviseletéhez","")            
            if "A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjához:" in data:
                data=data.replace("A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjához:","")
            if "Ezt a gyógyszert az Európai Gazdasági Térség tagállamaiban az alábbi neveken engedélyezték:" in data:
                data=data.split("Ezt a gyógyszert az Európai Gazdasági Térség tagállamaiban az alábbi neveken engedélyezték:")[0] 
            if "Nyilvántartási száma" in data:
                data=data.split("Nyilvántartási száma")[0]
            if "Nyilvántartási számai" in data:
                data=data.split("Nyilvántartási számai")[0]
            if "A betegtájékoztató legutóbbi felülvizsgálatának dátuma" in data:
                data=data.split("A betegtájékoztató legutóbbi felülvizsgálatának dátuma")[0]
            if "[logo'" in data:
                data=data.replace("[logo'","").strip()
            if "PVC/PVDC//Al buborékcsomagolásban" in data:
                data=data.replace("PVC/PVDC//Al buborékcsomagolásban","").strip()
            if "PVC/PE/PVDC//Al buborékcsomagolásban" in data:
                data=data.replace("PVC/PE/PVDC//Al buborékcsomagolásban","").strip()
            if "A forgalomba hozatali engedély száma" in data:
                data=data.replace("A forgalomba hozatali engedély száma","").strip()
            if "Kid 0,5 mg/ml oldatos orrspray" in data:
                data=data.replace("Kid 0,5 mg/ml oldatos orrspray","").strip()   

    return data  

In [227]:
#check what is shown under 'p' if there is no promo
len(searched_product_all[0].div.div.select('p'))

original_price=[]
promo_price=[]
for i in range(len(searched_product_all)):
    if len(searched_product_all[i].div.div.select('p')) !=0:
        original_price.append(cleanstring(str(searched_product_all[i].div.div.p.span.get_text())))
        promo_price.append(cleanstring(str(searched_product_all[i].div.div.p.next_sibling.next_sibling.span.get_text())))
    else: original_price.append("üres"); promo_price.append("üres")
        
original_price
promo_price

['üres']

In [228]:
#filter to a details url 
#searched_product_all[1].h2.a.get('href')

In [229]:
#generate the url list of the details pages
url_list_details=[]
for i in range(len(searched_product_all)):
    #collect href tags and save into url_list_details->will be used in the 2nd round of scraping
    searched_product_details_url=searched_product_all[i].h2.a.get('href')
    url_list_details.append( searched_product_details_url)
print(url_list_details)

['https://benu.hu/shop/zyrtec-10-mg-ml-belsoleges-oldatos-cseppek-20ml']


In [230]:
#check the length of the url list
len(url_list_details)

1

In [231]:
#generate soup from the html of the details pages
soup_list_details=[]

start = timer()

for i in range(len(url_list_details)):
    soup_list_details.append(BeautifulSoup(get(url_list_details[i]).text))
#print(up_list_details)
#print(len(up_list_details))

end = timer()

In [232]:
#check how much time generating the soup takes
print(end - start)#time in seconds

1.0302168999996866


In [233]:
#check length of the soup
len(soup_list_details)

1

In [234]:
#the <div class="product-shop"> includes the most of the product details however there are too many divs embeded and the required info placed not in directly neighbour divs. Therefore I soup them separately.
#check product category
searched_product_breadcrumbs_all=[]

start = timer()

#search for the last <li> tag which is the one with product, in this way the last category breadcrumb can be found as previous sibling
for i in range(len(soup_list_details)):
    product_breadcrumbs_temp=soup_list_details[i].find('div', {'class': 'breadcrumbs'})
    if product_breadcrumbs_temp != None:
        searched_product_breadcrumbs_all.append(soup_list_details[i].find('div', {'class': 'breadcrumbs'}).find('li', {'class': 'product'}).previous_sibling.previous_sibling.a.get_text('title'))
    else:searched_product_breadcrumbs_all.append("üres")
len(searched_product_breadcrumbs_all)
searched_product_breadcrumbs_all

['Allergia']

In [235]:
#product short description
searched_product_short_description_all=[]

start = timer()

for i in range(len(soup_list_details)):
    soup_of_short_description=soup_list_details[i].find('div', {'class': 'short-description'})
    if soup_of_short_description != None:
        searched_product_short_description_all.append(soup_list_details[i].find('div', {'class': 'short-description'}).div.get_text('p'))
    else:searched_product_short_description_all.append("üres")
len(searched_product_short_description_all)
searched_product_short_description_all

['A Zyrtec egy allergia elleni gyógyszer. A Zyrtec felnőttek, valamint 2 éves és afeletti gyermekek alábbi tüneteinek kezelésére javasolt: szezonális és egész éven át tartó allergiás nátha orr- és szemtüneteinek enyhítésére, krónikus csalánkiütés (krónikus idiopátiás urtikária) enyhítésére.']

In [236]:
#active agent
searched_product_active_agent_all=[]

for i in range(len(soup_list_details)):
    if soup_list_details[i].find('span', {'class': 'benu_hatoanyag'}) != None :
        if soup_list_details[i].find('span', {'class': 'benu_hatoanyag'}).get('data-content')!='':
            searched_product_active_agent_all.append(soup_list_details[i].find('span', {'class': 'benu_hatoanyag'}).get('data-content'))
        else: searched_product_active_agent_all.append("üres")
    else: searched_product_active_agent_all.append("üres")
len(searched_product_active_agent_all)
searched_product_active_agent_all


['cetirizinium chloratum']

In [237]:
#product availability
searched_product_availability_all=[]

for i in range(len(soup_list_details)):
    soup_of_availability=soup_list_details[i].find('div', {'class': 'availability in-stock'})
    if soup_of_availability != None:
        searched_product_availability_all.append(soup_of_availability.span.next_sibling.get_text())
    else:searched_product_availability_all.append("üres")
len(searched_product_availability_all)
searched_product_availability_all

['Készleten']

In [238]:
#unit price
searched_product_unit_price_all=[]

for i in range(len(soup_list_details)):
    product_unit_price=soup_list_details[i].find('p', {'class': 'unit-price'})
    spl_word=": "
    if product_unit_price != None:
        searched_product_unit_price_temp=(soup_list_details[i].find('p', {'class': 'unit-price'}).get_text())
        searched_product_unit_price_all.append(searched_product_unit_price_temp.partition(spl_word)[2])
    else:searched_product_unit_price_all.append("üres")

len(searched_product_unit_price_all)
searched_product_unit_price_all

['77,50 Ft/ml']

In [239]:
#Product can be payed with EP(Egészségpénztár) 
searched_product_ep_all=[]

for i in range(len(soup_list_details)):
    #product_ep=soup_list_details[i].find_all('div', {'class': 'termeklap-ikon'})    
    product_ep=soup_list_details[i].find_all('div', {'class': 'termeklap-ikon-text'})
    if len(product_ep)>1:
        searched_product_ep_all.append(product_ep[1].get_text())
    else: searched_product_ep_all.append("üres")

len( searched_product_ep_all)
searched_product_ep_all[1:5]   

[]

In [240]:
#Leaflet copy of the website
searched_product_ogyi=[]


for i in range(len(soup_list_details)):
    product_ogyi_v1=soup_list_details[i].find('div', {'class': 'WordSection1'})
    product_ogyi_v2=soup_list_details[i].find('div', {'class': 'Section1'})
    if product_ogyi_v1!= None:
        searched_product_ogyi.append(soup_list_details[i].find('div', {'class': 'WordSection1'}))
    elif product_ogyi_v2!= None:
        searched_product_ogyi.append(soup_list_details[i].find('div', {'class': 'Section1'}))
    else:searched_product_ogyi.append("üres")

#original code but accidently I found that on some urls the class name is 'Section 1'
# for i in range(len(soup_list_details)):
#     product_ogyi=soup_list_details[i].find('div', {'class': 'WordSection1'})
#     if product_ogyi != None:
#         searched_product_ogyi.append(product_ogyi.find_all('p', {'class': 'MsoNormal'}))
#     else:searched_product_ogyi.append("üres")

len(searched_product_ogyi)

1

In [241]:
#number of products where no leaflet copy is placed
no_available_leaflet_section=[]
counter=0
for i in range(len(searched_product_ogyi)):
    if searched_product_ogyi[i]=='üres':
        no_available_leaflet_section.append(url_list_details)
        counter=counter+1
no_available_leaflet_section
counter       
        

0

In [242]:
#Get Vendor, Manufactrer, Vendor&Manufacturer name and address, OGYI numbers, Leaflet release date
vendor_list=[]
manufacturer_list=[]
ogyi_list=[]
ad_list=[]

for i in range(len(searched_product_ogyi)):
    mystring = str(searched_product_ogyi[i]).replace('\n', ' ').replace('\\r', ' ').replace(u'\xa0',' ').replace("  "," ").replace(", ,",",").replace("</span></p>",",").replace("</p>",",")
    vendor=re.compile(r'(A forgalomba.*)(?![^<>]*>)') 
    vendor=vendor.findall(mystring)
    if 'Gyártó' in str(vendor):
        vendor_list.append(str(vendor).split('Gyártó')[0])
    elif 'A gyártó' in str(vendor):
        vendor_list.append(str(vendor).split('A gyártó')[0])
    else: vendor_list.append(str(vendor).split('OGYI')[0])
    manufacturer=re.compile(r'(Gyártó.*)(?![^<>]*>)') 
    manufacturer=manufacturer.findall(mystring)
    manufacturer_list.append(str(manufacturer).split('OGYI')[0])
    ogyi=re.compile(r'(OGYI.*)(?![^<>]*>)') 
    ogyi=ogyi.findall(mystring)
    ogyi_list.append(str(ogyi).split('A betegtájékoztató')[0])
    ad=re.compile(r'(A betegtájékoztató\r engedélyezésének dátuma.*)(?![^<>]*>)') 
    ad=ad.findall(mystring)
    ad_list.append(ad)
    
len(vendor_list)
len(manufacturer_list)
len(ogyi_list)
len(ad_list)

1

In [243]:
vendor_list

['[\'A forgalomba hozatali\\r engedély jogosultja és a gyártó</span></b>, <p class="MsoNormal"><span style="font-size:11.0pt"> , <p class="MsoNormal" style="line-height:13.0pt"><span style="font-size:11.0pt">UCB\\r Magyarország Kft., <p class="MsoNormal" style="line-height:13.0pt"><span style="font-size:11.0pt">H-1023\\r Budapest, <p class="MsoNormal" style="line-height:13.0pt"><span style="font-size:11.0pt">Árpád\\r fejedelem útja 26-28., <p class="MsoNormal" style="line-height:13.0pt"><span style="font-size:11.0pt">Magyarország, <p class="MsoNormal"><span style="font-size:11.0pt"> , <p class="MsoNormal"><span style="font-size:11.0pt">']

In [244]:
# for i in range(len(vendor_list)):
#     vendor_list[i]=vendor_list[i].replace("</span></p>",",").replace("</p>",",")
# vendor_list

In [245]:
#Remove html tags of vendor list
vendor_list_string=[]
for i in range(len(vendor_list)):
    if vendor_list[i]!='[]' or vendor_list[i]!='' or vendor_list[i]!=None:
        if cleanstring(removeunwantedparts(cleanstring(striphtml(str(vendor_list[i])))))!='':
            vendor_list_string.append(cleanstring(removeunwantedparts(cleanstring(striphtml(str(vendor_list[i]))))))
        else: vendor_list_string.append("üres")
    else: vendor_list_string.append("üres")  

vendor_list_string

['UCB Magyarország Kft., H-1023 Budapest, Árpád fejedelem útja 26-28., Magyarország']

In [246]:
#vendor's name
vendor_list_name=[]

for i in range(len(vendor_list_string)):
    if vendor_list_string[i]!="[]":
        vendor_list_name.append(vendor_list_string[i].split(",")[0])
    else: vendor_list_name.append("üres")
vendor_list_name

['UCB Magyarország Kft.']

In [247]:
#vendor's address
vendor_list_address=[]
for i in range(len(vendor_list_string)):
    if vendor_list_string[i]!="[]":
        if vendor_list_name[i] in vendor_list_string[i]:
            if cleanstring(vendor_list_string[i].replace(vendor_list_name[i],""))!='':
                vendor_list_address.append(cleanstring(vendor_list_string[i].replace(vendor_list_name[i],"")))
            else: vendor_list_address.append("üres")
    else: vendor_list_address.append("üres")
vendor_list_address
#len(vendor_list_address)

['H-1023 Budapest, Árpád fejedelem útja 26-28., Magyarország']

In [248]:
#Vendor and Manufacturer
vendor_and_manufacturer_1=[]
for i in range(len(searched_product_ogyi)):
    mystring = str(searched_product_ogyi[i]).replace('\n', ' ').replace('\\r', ' ').replace(u'\xa0',' ').replace("  "," ").replace(", ,",",").replace("</span></p>",",").replace("</p>",",")
    vm=re.compile(r'(A forgalomba hozatali engedély jogosultja és.+gyártó.*)(?![^<>]*>)') #ez megtalálja pontosan 2x az OGYI-t, de nem a teljes összetételt
    vm=vm.findall(mystring)
    vendor_and_manufacturer_1.append(vm)
len(vendor_and_manufacturer_1)
type(vendor_and_manufacturer_1)

list

In [249]:
#Remove html tags of vendor_and_manufacturer list and clean string
vendor_and_manufacturer_1_string=[]
for i in range(len(vendor_and_manufacturer_1)):
    if vendor_and_manufacturer_1[i]!=[]:
        vendor_and_manufacturer_1_string.append(cleanstring(removeunwantedparts(cleanstring(striphtml(str(vendor_and_manufacturer_1[i]))))))
    else: vendor_and_manufacturer_1_string.append("üres")
#vendor_and_manufacturer_1_string
vendor_and_manufacturer_1_string

['üres']

In [250]:
#vendor_and_manufacturer_name
vendor_and_manufacturer_name=[]
for i in range(len(vendor_and_manufacturer_1_string)):
    if vendor_and_manufacturer_1_string[i]!="üres":
        vendor_and_manufacturer_name.append(vendor_and_manufacturer_1_string[i].split(",")[0])
    else: vendor_and_manufacturer_name.append("üres")
#vendor_and_manufacturer_name
vendor_and_manufacturer_name

['üres']

In [251]:
#vendor_and_manufacturer_address
vendor_and_manufacturer_address=[]
for i in range(len(vendor_and_manufacturer_1_string)):
    if vendor_and_manufacturer_1_string[i]!="üres":
        if vendor_and_manufacturer_name[i] in vendor_and_manufacturer_1_string[i]:
            vendor_and_manufacturer_address.append(cleanstring(vendor_and_manufacturer_1_string[i].replace(vendor_and_manufacturer_name[i],'').split("OGYI")[0]))
        else: vendor_and_manufacturer_address.append("üres")
    else: vendor_and_manufacturer_address.append("üres")
vendor_and_manufacturer_address
#len(vendor_and_manufacturer_address)

['üres']

In [252]:
#Manufacturers
#Remove html tags of manufacturer_list items
manufacturer_list_string=[]
for i in range(len(manufacturer_list)):
    if manufacturer_list[i]!="[]":
        manufacturer_list_string.append(cleanstring(removeunwantedparts(cleanstring(striphtml(str(manufacturer_list[i]))))))
    else: manufacturer_list_string.append("üres")
manufacturer_list_string

['UCB Pharma SpA, 15 Via Praglia, 10044 Pianezza, Olaszország']

In [253]:
#split the manufacturer name and list it
#fixing Sanofi
for i in range(len(manufacturer_list_string)):
    manufacturer_list_string[i]=manufacturer_list_string[i].replace("SANOFI-AVENTIS Zrt.","Sanofi Aventis Zrt.")
manufacturer_list_string                                                                        

manufacturer_name=[]
for i in range(len(manufacturer_list_string)):
    if str(manufacturer_list_string[i]) != "üres":
        manufacturer_name.append(cleanstring(manufacturer_list_string[i].split(",")[0]))
    else: manufacturer_name.append("üres")
manufacturer_name

['UCB Pharma SpA']

In [254]:
#remove manufacturer name and only keep address section
manufacturer_address=[]
for i in range(len(manufacturer_list_string)):
    if manufacturer_list_string[i]!= "üres":
        if cleanstring(removeunwantedparts(cleanstring(manufacturer_list_string[i].replace(manufacturer_name[i],""))))!='':
            manufacturer_address.append(cleanstring(removeunwantedparts(cleanstring(manufacturer_list_string[i].replace(manufacturer_name[i],"")))))
        else: manufacturer_address.append("üres")
    else: manufacturer_address.append("üres")
manufacturer_address

['15 Via Praglia, 10044 Pianezza, Olaszország']

In [255]:
#OGYI number  - previous code, but my assumption that some OGYI IDs were missing as I used only the last 50 items
# #work with the last50 <p> in order to target the right OGYI section.Otherwise there are couple of mentioning of OGYI accross the whole text..
# searched_product_ogyi_last50=[]
# for i in range(len(searched_product_ogyi)):
#     if searched_product_ogyi[i]!=None:
#         searched_product_ogyi_last50.append(searched_product_ogyi[i][-50:])
#     else:searched_product_ogyi_last50.append("üres")
# len(searched_product_ogyi_last50)

In [256]:
# #Filter to OGYI - previous code, but my assumption that some OGYI IDs were missing as I used only the last 50 items
# ogyi_list=[]

# for i in range(len(searched_product)):
#     mystring = str(searched_product_ogyi[i]).replace('\n', ' ').replace(u'\xa0',' ').replace("\\r","").replace("  "," ").replace("\t"," ") 
#     ogyi=re.compile(r'(OGYI.*)(?![^<>]*>)') 
#     ogyi=ogyi.findall(mystring)
#     ogyi_list.append(str(ogyi))
    
# ogyi_list

In [257]:
#Remove html tags of ogyi_list items and clean the string
ogyi_list_string=[]
for i in range(len(ogyi_list)):
    if ogyi_list[i]!= "[]":
        ogyi_list_string.append(cleanstring(removeunwantedparts(striphtml(str(ogyi_list[i])))))
    else: ogyi_list_string.append("üres")    
len(ogyi_list_string)

1

In [258]:
#Remove unnecessary copy parts
for i in range(len(ogyi_list_string)):
    if ogyi_list_string[i]!='üres':
        if "," in ogyi_list_string[i]:
            ogyi_list_string[i]=cleanstring(re.sub("A betegtájékoztató.*|Betegtájékoztató.*|A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjához.*|Ezt a gyógyszert az Európai Gazdasági Térség tagállamaiban az alábbi neveken engedélyezték.*", " ", str(ogyi_list_string[i])))
len(ogyi_list_string)
#url_list_details[3]

1

In [259]:
#number of products that have no OGYI ID placed
counter=0
for i in range(len(ogyi_list_string)):
    if ogyi_list_string[i]!='üres':
        counter=counter+1
counter      

1

In [260]:
# I am trying to identify which OGYI number belongs to the actual product, based on the package info that I obtain from the decieve the benu prodct name with the originally searched product name
#souped_product_names[4].replace(" ","").lower().replace("mg","")

In [261]:
#searched_product_all[4].h2.a.get_text('title').strip().replace(" ","").lower()

In [262]:
#searched_product_all[4].h2.a.get_text('title').strip().replace(" ","").lower().replace(souped_product_names[4].replace(" ","").lower(),"")

In [263]:
#removing the original product name string
product_package=[]
for i in range(len(ogyi_list_string)):
    product_package.append(searched_product_all[i].h2.a.get_text('title').strip().replace(" ","").lower().replace(souped_product_names[i].replace(" ","").lower(),""))
product_package

['zyrtec10mg/mlbelsőlegesoldatoscseppek20ml']

In [264]:
#removing manually each not necessary strings and just keep the package info
for i in range(len(product_package)):
    product_package[i]=product_package[i].replace("acc200granulátum","").replace("acchot600mgporbelsőlegesoldathoz","").replace("+szájfecskendő","").replace("daedalonettavégbélkúpgyermeknek","").replace("dorithricinszopogatótablettaerdeigyümölcsízű","").replace("béresimmunherbalfilmtablettagyógyszer","").replace("loligripporbelsőlegesoldathoz","").replace("lolimarinehakid0,5mg/mloldatosorrspray","").replace("mucoplantdr.theisseukaliptuszkenőcs","").replace("neo-anginszopogatótablettacseresznyeízű","").replace("neo-angindoloszopogatótabletta","").replace("otrivinrapidmenthol1mg/mladagolóoldatosorrspray","").replace("rhinosprayplusorrspray","").replace("septoleteextra3mg/1mgszopogatótablettacitrom-bodzaízű","").replace("sinupretfortebevonttabletta","").replace("strepsilscitromízűcukormentesszopogatótabletta","").replace("(tantumlemon)","").replace("wicktriactincitromízűporbelsőlegesoldathoz","").replace("xilomarekid0,5mg/mloldatosorrspray","").replace("strepfencukormentes8,75mgszopogatótabletta","").replace("strepsilseperízűcukormentesszopogatótabletta","")
product_package

['zyrtec10mg/mlbelsőlegesoldatoscseppek20ml']

In [265]:
#how many OGYI-s can be identified? Only 147 which is poor result..This idea is dropped for now
ogyi_filtered_to_product=[]
ogyi_of_package_variations=[]
for i in range(len(ogyi_list_string)):
    for j in range(len(ogyi_list_string[i].split(","))):
        if str(searched_product_all[i].h2.a.get_text('title').strip().replace(" ","")).lower() in ogyi_list_string[i].split(",")[j].replace(" ","").lower():
            ogyi_filtered_to_product.append(ogyi_list_string[i].split(",")[j].replace(" ","").lower().replace(str(searched_product_all[i].h2.a.get_text('title').strip().replace(" ","")).lower(),""))
        elif product_package[i] in ogyi_list_string[i].split(",")[j].replace(" ","").lower():
            ogyi_filtered_to_product.append(ogyi_list_string[i].split(",")[j])
        else: ogyi_of_package_variations.append(ogyi_list_string[i].split(",")[j])
len(ogyi_filtered_to_product)
#ogyi_of_package_variations

0

In [266]:
len(ogyi_list_string[0].split(","))
# for i in range(len(ogyi_list_string)):
#     if str(ogyi_list_string[i])!='üres':
#         ogyi_list_string[i]=str(ogyi_list_string[i].split(","))
# ogyi_list_string[0]

1

In [267]:
ogyi_list_string

['OGYI-T-4491/01']

In [268]:
#how many OGYI-s can be identified? Only 147 which is poor result..This idea is dropped for now
counter=0
counter_list=0
counter_identified_package=0
counter_identified_package_l2=0
ogyi_filtered=[]
for i in range(len(ogyi_list_string)):
    if ogyi_list_string[i]!="üres":
        #print("nem üres")
        if "," in str(ogyi_list_string[i]).replace(" ",""):
            counter_list=counter_list+1
            if str(product_package[i]).replace(" ","").split("x")[0] in str(ogyi_list_string[i]).replace(" ",""):
                for j in range(len(ogyi_list_string[i].split(","))):
                    if str(product_package[i]).replace(" ","").split("x")[0] in str(ogyi_list_string[i].split(",")[j]).replace(" ",""):
                        counter_identified_package_l2=counter_identified_package_l2+1
                        ogyi_filtered.append(str(ogyi_list_string[i].split(",")[j]))
            else:ogyi_filtered.append(ogyi_list_string[i])
    else:ogyi_filtered.append("üres")
                        
counter_list    
# counter_identified_package
# counter_identified_package_l2
len(ogyi_filtered)
#ogyi_filtered

0

In [269]:
merged=pd.DataFrame([ogyi_list_string,ogyi_filtered])
merged=merged.transpose(copy=False)
merged

,0,1
0,OGYI-T-4491/01,None


In [270]:
# create excel writer object
writer = pd.ExcelWriter('benu_ogyi_teszt_v2.xlsx')
# write dataframe to excel
merged.to_excel(writer)
# save the excel
writer.save()

In [271]:
#Leaflet release date
#the original compiling had to be revised based on OGYI's splitting words. I took the common part of those words.
ad_list_e1=[]

for i in range(len(searched_product_ogyi)):
    mystring = str(searched_product_ogyi[i]).replace('\n', ' ').replace(u'\xa0',' ').replace("\\r","").replace("  "," ").replace("\t"," ") 
    ad=re.compile(r'(.etegtájékoztató engedélyezés.*)(?![^<>]*>)') 
    ad=ad.findall(mystring)
    ad_list_e1.append(str(ad))
#ad_list_e1
len(ad_list_e1)

1

In [272]:
#Remove html tags of ad_list items
ad_list_string=[]
for i in range(len(ad_list_e1)):
    if ad_list_e1[i]!= "[]":
        ad_list_string.append(cleanstring(striphtml(str(ad_list_e1[i]))).replace("']","").strip())
    else: ad_list_string.append("üres")
        
ad_list_string

['betegtájékoztató engedélyezésének dátuma 2009. április 28.']

In [273]:
#Split out only the 'dates'
ad_list_date=[]
for i in range(len(ad_list_string)):
    if ad_list_string[i]!="üres":
        #ad_list_string[i]=ad_list_string[i].replace(",",":")
        ad_list_date.append(cleanstring(str(ad_list_string[i].split("dátuma")[1].strip())))
    else: ad_list_date.append("üres")
ad_list_date

['2009. április 28.']

In [274]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y")
print("date and time =", dt_string)	

now = 2021-02-12 12:28:51.589210
date and time = 12/02/2021


In [275]:
benu_search_result_all=pd.DataFrame(columns=['ogyi_product_name',
                                'product_name',
                                'package',
                                'price',
                                'promo_price',
                                'original_price',
                                'unit_price',
                                'ep',
                                'availability',
                                'product_category',
                                'active_agent',
                                'short_description',
                                'ogyi',
                                'ogyi_date',
                                'vendor_name',
                                'vendor_address',
                                'manufacturer_name',
                                'manufacturer_address',
                                'vendor_and_manufacturer_name',
                                'vendor_and_manufacturer_address',
                                'date'])

benu_search_result_all['ogyi_product_name']=souped_product_names
benu_search_result_all['product_name']=benu_product_name
benu_search_result_all['product_name']
benu_search_result_all['package']=product_package
benu_search_result_all['price']=price
benu_search_result_all['promo_price']=promo_price
benu_search_result_all['original_price']=original_price
benu_search_result_all['unit_price']=searched_product_unit_price_all
benu_search_result_all['ep']=searched_product_ep_all
benu_search_result_all['availability']=searched_product_availability_all
benu_search_result_all['product_category']=searched_product_breadcrumbs_all
benu_search_result_all['active_agent']=searched_product_active_agent_all
benu_search_result_all['short_description']=searched_product_short_description_all
benu_search_result_all['ogyi']=ogyi_list_string
benu_search_result_all['ogyi_date']=ad_list_date
benu_search_result_all['vendor_name']=vendor_list_name
benu_search_result_all['vendor_address']=vendor_list_address
benu_search_result_all['manufacturer_name']=manufacturer_name
benu_search_result_all['manufacturer_address']=manufacturer_address
benu_search_result_all['vendor_and_manufacturer_name']=vendor_and_manufacturer_name
benu_search_result_all['vendor_and_manufacturer_address']=vendor_and_manufacturer_address
benu_search_result_all['date']=dt_string

benu_search_result_all

,ogyi_product_name,product_name,package,price,promo_price,original_price,unit_price,ep,availability,product_category,...,short_description,ogyi,ogyi_date,vendor_name,vendor_address,manufacturer_name,manufacturer_address,vendor_and_manufacturer_name,vendor_and_manufacturer_address,date
0,ACC 100 mg granulátum gyermekeknek,Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml,zyrtec10mg/mlbelsőlegesoldatoscseppek20ml,1549,üres,üres,"77,50 Ft/ml",Egészségpénztári kártyára vásárolható,Készleten,Allergia,...,A Zyrtec egy allergia elleni gyógyszer. A Zyrt...,OGYI-T-4491/01,2009. április 28.,UCB Magyarország Kft.,"H-1023 Budapest, Árpád fejedelem útja 26-28., ...",UCB Pharma SpA,"15 Via Praglia, 10044 Pianezza, Olaszország",üres,üres,12/02/2021


In [276]:
# create excel writer object
writer = pd.ExcelWriter('benu_ogyi_teszt.xlsx')
# write dataframe to excel
benu_search_result_all.to_excel(writer)
# save the excel
writer.save()

In [277]:
#drop duplicated lines
benu_search_result_all.drop_duplicates(subset="product_name", keep="first", inplace=True)
len(benu_search_result_all)

1

In [278]:
# Create an empty list 
row_list =[] 
  
# Iterate over each row 
for index, rows in benu_search_result_all.iterrows(): 
    # Create list for the current row 
    my_list =[rows.ogyi_product_name,rows.product_name,rows.package, rows.price, rows.promo_price, rows.original_price, rows.unit_price, rows.ep, rows.availability, rows.product_category,rows.active_agent, rows.short_description, rows.ogyi, rows.ogyi_date, rows.vendor_name, rows.vendor_address, rows.manufacturer_name, rows.manufacturer_address, rows.vendor_and_manufacturer_name, rows.vendor_and_manufacturer_address, rows.date] 
      
    # append the list to the final list 
    row_list.append(my_list) 
print(row_list)

[['ACC 100 mg granulátum gyermekeknek', 'Zyrtec 10 mg/ml belsőleges oldatos cseppek 20ml', 'zyrtec10mg/mlbelsőlegesoldatoscseppek20ml', '1549', 'üres', 'üres', '77,50 Ft/ml', 'Egészségpénztári kártyára vásárolható', 'Készleten', 'Allergia', 'cetirizinium chloratum', 'A Zyrtec egy allergia elleni gyógyszer. A Zyrtec felnőttek, valamint 2 éves és afeletti gyermekek alábbi tüneteinek kezelésére javasolt: szezonális és egész éven át tartó allergiás nátha orr- és szemtüneteinek enyhítésére, krónikus csalánkiütés (krónikus idiopátiás urtikária) enyhítésére.', 'OGYI-T-4491/01', '2009. április 28.', 'UCB Magyarország Kft.', 'H-1023 Budapest, Árpád fejedelem útja 26-28., Magyarország', 'UCB Pharma SpA', '15 Via Praglia, 10044 Pianezza, Olaszország', 'üres', 'üres', '12/02/2021']]


In [279]:
#insert my data into the database
sql='INSERT INTO dbo.benu_raw_data(ogyi_product_name,product_name, package,price,promo_price,original_price,unit_price,ep,availability,product_category,active_agent,short_description,ogyi,ogyi_date,vendor_name,vendor_address,manufacturer_name,manufacturer_address,vendor_and_manufacturer_name,vendor_and_manufacturer_address,date) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'

for row in row_list:
    #print(row)
    values=(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16],row[17],row[18],row[19],row[20])
    #print(values)
    cursor.execute(sql, values)
    
#cursor.execute(sql, values)   
conn.commit()   

In [280]:
#read back the content of the database
cursor.execute('SELECT * from dbo.benu_raw_data')

In [281]:
#print out the data from the database
for row in cursor:
    print(row)

('ACC 100 mg granulátum gyermekeknek                                                                                                                                                                                                                             ', 'ACC 100 mg granulátum gyermekeknek 30x3g', '30x3g', '1549', 'üres', 'üres', '51,60 Ft/db', 'Egészségpénztári kártyára vásárolható', 'Készleten', 'Köhögéscsillapító', 'acetylcysteinum', 'Az ACC 100 mg granulátum gyermekeknek, amelynek hatóanyaga az acetilcisztein, a légutakban levő sűrű viszkózus váladék oldására alkalmas. Fokozott viszkozitású nyákelválasztással, köhögéssel járó légúti betegségek váladékoldást elősegítő terápiájában javallt.', 'OGYI-T-2021/01 ACC 100 mg granulátum gyermekeknek 20x3g, OGYI-T-2021/02 ACC 100 mg granulátum gyermekeknek 30x 3g', 'üres', 'Sandoz Hungária Kft.', 'Bartók Béla út 43-47., 1114 Budapest, Magyarország', 'Salutas Pharma Gmbh', 'Otto-von-Guericke Allee 1, D-39179 Barleben, Németország, Hexal 

('STREPSILS citromízû cukormentes szopogató tabletta                                                                                                                                                                                                             ', 'Strepsils citromízű cukormentes szopogató tabletta 24x', '24x', '2299', 'üres', 'üres', '95,80 Ft/db', 'Egészségpénztári kártyára vásárolható', 'Készleten', 'Torokfertőtlenítők', 'alcoholum dichlorobenzylicum', 'Antibakteriális, gomba és vírusellenes hatású. csillapítja a fájdalmat már 5 perc elteltével.', 'OGYI-T-4519/02 (24×)', 'üres', 'Reckitt Benckiser Healthcare International Ltd.', '103-105 Bath Road, Slough, Berkshire SL1 3UH, Egyesült Királyság', 'Reckitt Benckiser Healthcare International Limited', 'Nottingham Site, Thane Road Nottingham, NG90 2DB, Egyesült Királyság,:, Reckitt Benckiser Kft., 1113 Budapest, Bocskai út 134-146.,Tel: 06 1 880-1870, Fax: 06 1 250-8398, E-mail: gyogyszer@reckittbenckiser.com', 'üres', 'üres

('CETIRIZIN 1 A PHARMA 10 mg filmtabletta                                                                                                                                                                                                                        ', 'Cetirizin 1a Pharma 10 mg filmtabletta 10x', '10x', 'üres', 'üres', 'üres', '0,00 Ft/', 'Egészségpénztári kártyára vásárolható', 'Készleten', 'Allergia', 'cetirizinium chloratum', 'A Cetirizin 1a Pharma allergia ellenes gyógyszer. Segíti az allergiás reakciók és tünetek megszüntetését. Alkalmas a viszketés és a csalánkiütés enyhítésére is.', 'OGYI-T-10390/01 (7x), OGYI-T-10390/02 (10x), OGYI-T-10390/03 (30x), OGYI-T-10390/04 (60x)', 'üres', '1 A Pharma Gmbh', 'Keltenring 1+3, 82041 Oberhaching, Németország', 'Salutas Pharma Gmbh', 'Otto-von-Guericke-Allee 1, 39179 Barleben, Németország', 'üres', 'üres', '25/07/2020')
('MUCOFREE 30 mg/5 ml szirup                                                                                      

('WICK TRIACTIN citrom ízû por belsőleges oldathoz', 'Wick TriActin citrom ízű por belsőleges oldathoz 10x', '10x', '2599', 'üres', 'üres', '259,90 Ft/db', 'Egészségpénztári kártyára vásárolható', 'Készleten', 'Komplex készítmények, forró italporok', 'paracetamolum', 'A TriActin Wick citrom ízű por belsőleges oldathoz a megfázás és az influenza tüneteinek enyhítésére szolgál. Három hatóanyagot tartalmaz: A paracetamol jól ismert fájdalomcsillapító. A különböző fájdalmak, így a fejfájás és a torokfájás megszüntetésére és a láz csillapítására alkalmazható. A gvajfenezin (köhögéscsillapító) fellazítja a váladékot és enyhíti a hurutos köhögést. A fenilefrin-hidroklorid (orrdugulás elleni szer) csökkenti az orrjárat duzzanatát, így enyhíti az orrdugulást.', 'OGYI-T-21591/01-02', 'üres', 'Wick Pharma', 'Zweigniederlassung der Procter & Gamble GmbH, Sulzbacher Str. 40-50, D-65824 Schwalbach am Taunus, Németország', 'Wrafton Laboratories Limited', 'Braunton, Devon, EX33 2DL,Nagy-Britannia', 'ü

In [282]:
#close cursor&connection
cursor.close()
conn.close()